In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
# Set up folds
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)

In [3]:
!head loss-5.7552val-2.9950_val

id,scalar_coupling_constant
0,83.50312805175781
1,-2.245182991027832
2,-11.567841529846191
3,-11.566396713256836
4,3.245485305786133
5,12.714962005615234
6,3.298550844192505
7,83.466064453125
8,-2.2379026412963867


In [2]:
subs = ['loss-5.7552val-2.9950']

-rw-r--r-- 1 root root 11847733 Aug  3 20:10 loss-5.7552val-2.9950_val


In [ ]:
train_x = None
for sub in subs:
    df = pd.read_csv('%s_val' % sub)
    df = df.drop(['id'], axis=1)
    if train_df is None:
        train_x = df
        train_x['scalar_coupling_constant_%s' % sub] = df['scalar_coupling_constant']
    else:
        train_x['scalar_coupling_constant_%s' % sub] = df['scalar_coupling_constant']
train_x = train_x.drop(['scalar_coupling_constant'], axis=1)
train_x.head()

In [ ]:
train_y = pd.read_csv('validation_targets')
train_y = train_y.drop(['id'], axis=1)['scalar_coupling_constant']
train_y

In [ ]:
MAX_ROUNDS = 650
model = CatBoostRegressor(n_estimators=200,
                          loss_function='MAE',
                          eval_metric='RMSE',
                          #iterations = MAX_ROUNDS,
                          cat_features=None)

In [ ]:
y = train_y
X = train_x

In [ ]:
#model.fit(dtrain,
#          ltrain,
#          use_best_model=True,
#          eval_set=(dval, lval),
#          silent=True,
#          plot=True)

In [ ]:
# Run CV
# https://www.kaggle.com/aharless/simple-catboost-cv-lb-281
OPTIMIZE_ROUNDS = True


for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
    print( "\nFold ", i)
    
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        fit_model = model.fit( X_train, y_train, 
                               eval_set=[X_valid, y_valid],
                               use_best_model=True
                             )
        print( "  N trees = ", model.tree_count_ )
    else:
        fit_model = model.fit( X_train, y_train )
        
    # Generate validation predictions for this fold
    pred = fit_model.predict(X_valid)
    #print( "  Gini = ", eval_gini(y_valid, pred) )
    #y_valid_pred.iloc[test_index] = pred
    
    # Accumulate test set predictions
    #y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
#y_test_pred /= K  # Average test set predictions
